**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 4 - Open Science Platform - Cross link User and Project accounts

1 - Load required modules

In [1]:
from new_helper import *
from super_helper import *
from loguru import logger

2 - Set variables and values

In [2]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 0

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [3]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-02 20:04:45,892 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-02 20:04:45.893 | INFO     | __main__:<module>:3 - b'6db5738a5c634925422b433313fe74ce671f54fdbd480d024e1c60f7f22e46b8'


5 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` (see item 2) with a entry number related to the position of an existing object in `datasets/accounts.json` and `datasets/projects.json`. Example: `json_ld_index = 2`, selects the second object in each json file.

6 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [4]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-02-02 20:04:45.903 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-02 20:04:45.913 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'6db5738a5c634925422b433313fe74ce671f54fdbd480d024e1c60f7f22e46b8'
2025-02-02 20:04:45.913 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: BBD42BBFF4F6911A42D249C85FA8F7BB246E6130
2025-02-02 20:04:45.914 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-02 20:04:45.915 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-02 20:04:45,932 - INFO - None
2025-02-02 20:04:45,935 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-02-02 20:04:46,656 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-02 20:04:46,664 - INFO - ('COMMITTED', 5, 0)
2025-02-02 20:04:46.666 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-02 20:04:46.686 | INFO     | integration_helpers:get_engine_receipts_result:146 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-02 20:04:46.687 | DEBUG    | integration_helpers:get_engine_receipts_result:147 - Result decoded string:                                                                                                
2025-02-02 20:04:46.688 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-02 20:04:46.690 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "set_account_detail"
2025-02-02 20:04:46.691 | DEBUG    | integration_helpers

7 - Queries Iroha 1 for the User account and exposes the variable `project_id` from user details.

In [5]:
#Query - GetAccountDetail

from iroha_helper import get_account_detail

user_details = get_account_detail(f"{user_account['account_id']}")

# logger.info(f'User Account id = {user_account}, {user_details}')

# # Process the account details response in JSON format
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object

# # Exposes the project_id 
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(f"Project Id = {project_id}")

2025-02-02 20:04:49.765 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account_detail"
2025-02-02 20:04:49,787 - INFO - User Account id = compassionate_khorana@test, { "admin@test" : { "account_metadata_cid" : "QmP4fwBk5yhqR2kYvSgVg7K46Fb1PwNe6Zgy83srGdP7D7", "linked_project" : "84131@test" } }
2025-02-02 20:04:49.788 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account_detail"
2025-02-02 20:04:49.790 | INFO     | __main__:<module>:14 - Project Id = 84131@test


8 - Saves `project_id` in the local file sytem for use in Notebook 4

In [6]:
# Dumping the variable to JSON
dump_variable(project_id, "Project Id")

2025-02-02 20:04:49,815 - INFO - Variable 'Project Id' successfully dumped to temp/Project Id.json
